In [1]:
# 1. Import Libraries
import os
import sys
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from pathlib import Path
from typing import List, Tuple
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as TF

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

# Set random seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)
print("✓ Random seeds set")

/Users/Gabriel/dev/konkuk/IA/project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch version: 2.9.1
CUDA available: False
✓ Random seeds set


In [2]:
# 2. Dataset Class (GoProDataset)
# Copied from src/dataset.py

class GoProDataset(Dataset):
    """
    GoPro Image Deblurring Dataset.
    
    Features:
    - On-the-fly random cropping (256x256 patches from 1280x720 images)
    - Random horizontal flip augmentation
    - Normalization to [0, 1] range
    - Optional full-resolution mode (no cropping)
    """
    
    def __init__(
        self,
        blur_paths: List[str],
        sharp_paths: List[str],
        patch_size: int = 256,
        is_train: bool = True,
        full_image: bool = False
    ):
        assert len(blur_paths) == len(sharp_paths), "Mismatch between blur and sharp image counts"
        
        self.blur_paths = blur_paths
        self.sharp_paths = sharp_paths
        self.patch_size = patch_size
        self.is_train = is_train
        self.full_image = full_image
        
    def __len__(self) -> int:
        return len(self.blur_paths)
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Load and process a single image pair.
        """
        # Load images as PIL
        blur_img = Image.open(self.blur_paths[idx]).convert('RGB')
        sharp_img = Image.open(self.sharp_paths[idx]).convert('RGB')
        
        # Apply cropping only if not using full images
        if not self.full_image:
            # Apply cropping (random for train, center for val/test)
            if self.is_train:
                # Random crop - same location for both images
                i, j, h, w = self._get_random_crop_params(blur_img)
            else:
                # Center crop for validation/test
                i, j, h, w = self._get_center_crop_params(blur_img)
            
            blur_img = TF.crop(blur_img, i, j, h, w)
            sharp_img = TF.crop(sharp_img, i, j, h, w)
        
        # Random horizontal flip (only during training)
        if self.is_train and random.random() > 0.5:
            blur_img = TF.hflip(blur_img)
            sharp_img = TF.hflip(sharp_img)
        
        # Convert to tensor and normalize to [0, 1]
        blur_tensor = TF.to_tensor(blur_img)  # Converts uint8 [0, 255] to float [0, 1]
        sharp_tensor = TF.to_tensor(sharp_img)
        
        return blur_tensor, sharp_tensor
    
    def _get_random_crop_params(self, img: Image.Image) -> Tuple[int, int, int, int]:
        width, height = img.size
        if width < self.patch_size or height < self.patch_size:
            raise ValueError(f"Image size ({width}x{height}) is smaller than patch size ({self.patch_size})")
        
        top = random.randint(0, height - self.patch_size)
        left = random.randint(0, width - self.patch_size)
        return top, left, self.patch_size, self.patch_size
    
    def _get_center_crop_params(self, img: Image.Image) -> Tuple[int, int, int, int]:
        width, height = img.size
        top = (height - self.patch_size) // 2
        left = (width - self.patch_size) // 2
        return top, left, self.patch_size, self.patch_size


def get_image_pairs(root_dir: str, split: str = 'train') -> Tuple[List[str], List[str]]:
    """Recursively find blur-sharp image pairs in the dataset."""
    blur_paths = []
    sharp_paths = []
    
    split_dir = os.path.join(root_dir, split)
    
    if not os.path.exists(split_dir):
        # Fallback for Colab if path is slightly different or just to warn
        print(f"Warning: Directory {split_dir} does not exist. Check your data_root.")
        return [], []
    
    # Walk through sequence folders
    for sequence_folder in sorted(os.listdir(split_dir)):
        sequence_path = os.path.join(split_dir, sequence_folder)
        if not os.path.isdir(sequence_path):
            continue
        
        blur_dir = os.path.join(sequence_path, 'blur')
        sharp_dir = os.path.join(sequence_path, 'sharp')
        
        if os.path.exists(blur_dir) and os.path.exists(sharp_dir):
            images = sorted(os.listdir(blur_dir))
            for img_name in images:
                if img_name.endswith('.png'):
                    blur_paths.append(os.path.join(blur_dir, img_name))
                    sharp_paths.append(os.path.join(sharp_dir, img_name))
    
    return blur_paths, sharp_paths

In [3]:
# 3. Model Architecture (SimpleUNet)
# Copied from src/models/unet.py

class ConvBlock(nn.Module):
    """Basic convolutional block: Conv → InstanceNorm → LeakyReLU."""
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, padding_mode='reflect', bias=False)
        self.norm = nn.InstanceNorm2d(out_channels, affine=True)
        self.activation = nn.LeakyReLU(0.2, inplace=True)
    
    def forward(self, x):
        return self.activation(self.norm(self.conv(x)))

class ResBlock(nn.Module):
    """Residual Block: x + Conv(Norm(Act(Conv(Norm(Act(x))))))"""
    def __init__(self, channels):
        super().__init__()
        self.conv1 = nn.Conv2d(channels, channels, 3, 1, 1, padding_mode='reflect', bias=False)
        self.norm1 = nn.InstanceNorm2d(channels, affine=True)
        self.act = nn.LeakyReLU(0.2, inplace=True)
        self.conv2 = nn.Conv2d(channels, channels, 3, 1, 1, padding_mode='reflect', bias=False)
        self.norm2 = nn.InstanceNorm2d(channels, affine=True)
        
    def forward(self, x):
        residual = x
        out = self.act(self.norm1(self.conv1(x)))
        out = self.norm2(self.conv2(out))
        return out + residual

class EncoderBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = ConvBlock(in_channels, out_channels)
        self.res = ResBlock(out_channels)
        self.downsample = nn.Conv2d(out_channels, out_channels, 4, 2, 1, padding_mode='reflect', bias=False)
    
    def forward(self, x):
        x = self.conv(x)
        features = self.res(x)
        downsampled = self.downsample(features)
        return features, downsampled

class DecoderBlock(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels):
        super().__init__()
        self.upsample = nn.ConvTranspose2d(in_channels, in_channels, 4, 2, 1, bias=False)
        self.conv = ConvBlock(in_channels + skip_channels, out_channels)
        self.res = ResBlock(out_channels)
    
    def forward(self, x, skip):
        x = self.upsample(x)
        x = torch.cat([x, skip], dim=1)
        x = self.conv(x)
        x = self.res(x)
        return x

class SimpleUNet(nn.Module):
    """
    Simple U-Net for image deblurring with Global Residual Learning.
    """
    def __init__(self, in_channels=3, out_channels=3, global_residual=True):
        super().__init__()
        self.global_residual = global_residual
        
        # Encoder
        self.encoder1 = EncoderBlock(in_channels, 64)
        self.encoder2 = EncoderBlock(64, 128)
        self.encoder3 = EncoderBlock(128, 256)
        self.encoder4 = EncoderBlock(256, 512)
        
        # Bottleneck
        self.bottleneck = nn.Sequential(ConvBlock(512, 512), ConvBlock(512, 512))
        
        # Decoder
        self.decoder4 = DecoderBlock(512, 512, 256)
        self.decoder3 = DecoderBlock(256, 256, 128)
        self.decoder2 = DecoderBlock(128, 128, 64)
        self.decoder1 = DecoderBlock(64, 64, 64)
        
        # Output
        self.output = nn.Conv2d(64, out_channels, kernel_size=1)

        # Zero Initialization for Residual Learning
        if global_residual:
            nn.init.constant_(self.output.weight, 0)
            if self.output.bias is not None:
                nn.init.constant_(self.output.bias, 0)
    
    def forward(self, x):
        input_image = x if self.global_residual else None
        
        skip1, x = self.encoder1(x)
        skip2, x = self.encoder2(x)
        skip3, x = self.encoder3(x)
        skip4, x = self.encoder4(x)
        
        x = self.bottleneck(x)
        
        x = self.decoder4(x, skip4)
        x = self.decoder3(x, skip3)
        x = self.decoder2(x, skip2)
        x = self.decoder1(x, skip1)
        
        x = self.output(x)
        
        if self.global_residual:
            x = x + input_image
        
        return x

In [4]:
# 4. Metrics
# Copied from src/utils/metrics.py

class PSNRMetric:
    """Accumulates PSNR values across batches."""
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.total_psnr = 0.0
        self.count = 0
        
    def update(self, pred, target):
        # Calculate PSNR for current batch
        with torch.no_grad():
            # Clamp to [0, 1]
            pred = torch.clamp(pred, 0, 1)
            target = torch.clamp(target, 0, 1)
            
            mse = F.mse_loss(pred, target, reduction='none').mean(dim=[1, 2, 3])
            # Handle perfect matches
            mse[mse == 0] = 1e-10
            
            psnr = 10 * torch.log10(1.0 / mse)
            self.total_psnr += psnr.sum().item()
            self.count += pred.size(0)
            
    def compute(self):
        return self.total_psnr / self.count if self.count > 0 else 0.0

def calculate_psnr(img1, img2):
    """Helper for single batch PSNR calculation."""
    with torch.no_grad():
        mse = F.mse_loss(img1, img2)
        if mse == 0: return 100.0
        return 10 * torch.log10(1.0 / mse)

In [5]:
# 5. Configuration
class Config:
    """Training configuration"""
    
    # Data
    data_root = './data'    # CHANGE THIS PATH ON COLAB (e.g., '/content/drive/MyDrive/GoPro')
    patch_size = 256        # Crop size for training
    batch_size = 8          # Batch size
    num_workers = 2         # Lower workers for Colab safety
    
    # Model
    in_channels = 3
    out_channels = 3
    
    # Training
    num_epochs = 100        
    learning_rate = 2e-4    # Stable LR for residual learning
    weight_decay = 1e-4     
    
    # Loss function
    loss_type = 'charbonnier'
    
    # Scheduler
    scheduler_patience = 10
    scheduler_factor = 0.5
    
    # Checkpointing
    save_dir = './experiments'
    experiment_name = 'simple_unet_colab'
    
    # Early stopping
    early_stop_patience = 20
    
    # Sanity check mode
    sanity_check = False    # Set to True to overfit on 10 images first

config = Config()

# Create save directory
os.makedirs(config.save_dir, exist_ok=True)
experiment_dir = os.path.join(config.save_dir, config.experiment_name)
os.makedirs(experiment_dir, exist_ok=True)

print("Configuration:")
print(f"  Data Root: {config.data_root}")
print(f"  Batch Size: {config.batch_size}")
print(f"  Learning Rate: {config.learning_rate}")
print(f"  Sanity Check: {config.sanity_check}")

Configuration:
  Data Root: ./data
  Batch Size: 8
  Learning Rate: 0.0002
  Sanity Check: False


In [6]:
# 6. Device Setup
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using CUDA: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device('cpu')
    print("Using CPU (training will be slow!)")

Using CPU (training will be slow!)


In [7]:
# 7. Loss Function
class CharbonnierLoss(nn.Module):
    """Smooth L1 Loss: sqrt(diff^2 + epsilon^2)"""
    def __init__(self, epsilon=1e-3):
        super().__init__()
        self.epsilon = epsilon
    
    def forward(self, pred, target):
        diff = pred - target
        loss = torch.sqrt(diff * diff + self.epsilon * self.epsilon)
        return loss.mean()

def get_loss_function(loss_type='l1'):
    if loss_type == 'l1': return nn.L1Loss()
    elif loss_type == 'mse': return nn.MSELoss()
    elif loss_type == 'charbonnier': return CharbonnierLoss()
    else: raise ValueError(f"Unknown loss type: {loss_type}")

criterion = get_loss_function(config.loss_type)
print(f"Loss function: {config.loss_type.upper()}")

Loss function: CHARBONNIER


In [8]:
# 8. Load Data
print("Loading data...")
train_blur, train_sharp = get_image_pairs(config.data_root, 'train')
test_blur, test_sharp = get_image_pairs(config.data_root, 'test')

print(f"Found {len(train_blur)} training pairs")
print(f"Found {len(test_blur)} test pairs")

if len(train_blur) == 0:
    print("⚠️ NO DATA FOUND! Please check Config.data_root path.")
else:
    # Sanity check mode logic
    if config.sanity_check:
        train_blur = train_blur[:10]
        train_sharp = train_sharp[:10]
        test_blur = train_blur 
        test_sharp = train_sharp
        print(f"🔬 SANITY CHECK MODE: Using {len(train_blur)} samples")

    # Create datasets
    train_dataset = GoProDataset(
        train_blur, train_sharp, 
        patch_size=config.patch_size, 
        is_train=not config.sanity_check
    )

    val_dataset = GoProDataset(
        test_blur, test_sharp, 
        patch_size=config.patch_size, 
        is_train=False
    )

    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.batch_size,
        shuffle=not config.sanity_check,
        num_workers=config.num_workers,
        pin_memory=True if torch.cuda.is_available() else False
    )

    val_loader = DataLoader(
        val_dataset,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=config.num_workers,
        pin_memory=True if torch.cuda.is_available() else False
    )
    print("✓ Data loaded successfully")

Loading data...
Found 2103 training pairs
Found 1111 test pairs
✓ Data loaded successfully


In [9]:
# 9. Initialize Model & Optimizer
model = SimpleUNet(
    in_channels=config.in_channels,
    out_channels=config.out_channels,
    global_residual=True
).to(device)

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=config.learning_rate,
    weight_decay=config.weight_decay
)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=config.scheduler_factor,
    patience=config.scheduler_patience,
)

print(f"Model parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print("✓ Model initialized")

Model parameters: 28,480,003
✓ Model initialized


In [10]:
# 10. Training Functions
def train_one_epoch(model, train_loader, criterion, optimizer, device, epoch):
    model.train()
    running_loss = 0.0
    psnr_metric = PSNRMetric()
    
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]")
    
    for blur, sharp in pbar:
        blur, sharp = blur.to(device), sharp.to(device)
        
        output = model(blur)
        loss = criterion(output, sharp)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        running_loss += loss.item()
        psnr_metric.update(output.detach(), sharp)
        
        pbar.set_postfix({'loss': f"{loss.item():.4f}", 'psnr': f"{psnr_metric.compute():.2f}"})
    
    return running_loss / len(train_loader), psnr_metric.compute()

def validate(model, val_loader, criterion, device, epoch):
    model.eval()
    running_loss = 0.0
    psnr_metric = PSNRMetric()
    
    pbar = tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]  ")
    
    with torch.no_grad():
        for blur, sharp in pbar:
            blur, sharp = blur.to(device), sharp.to(device)
            output = model(blur)
            loss = criterion(output, sharp)
            
            running_loss += loss.item()
            psnr_metric.update(output, sharp)
            
            pbar.set_postfix({'loss': f"{loss.item():.4f}", 'psnr': f"{psnr_metric.compute():.2f}"})
    
    return running_loss / len(val_loader), psnr_metric.compute()

In [ ]:
# 11. Main Training Loop
history = {'train_loss': [], 'train_psnr': [], 'val_loss': [], 'val_psnr': [], 'lr': []}
best_psnr = 0.0
epochs_without_improvement = 0

print("Starting training...")
for epoch in range(config.num_epochs):
    # Train
    train_loss, train_psnr = train_one_epoch(model, train_loader, criterion, optimizer, device, epoch)
    
    # Validate
    val_loss, val_psnr = validate(model, val_loader, criterion, device, epoch)
    
    # Scheduler
    scheduler.step(val_psnr)
    current_lr = optimizer.param_groups[0]['lr']
    
    # History
    history['train_loss'].append(train_loss)
    history['train_psnr'].append(train_psnr)
    history['val_loss'].append(val_loss)
    history['val_psnr'].append(val_psnr)
    history['lr'].append(current_lr)
    
    print(f"Epoch {epoch+1}: Train PSNR={train_psnr:.2f}dB | Val PSNR={val_psnr:.2f}dB | LR={current_lr:.2e}")
    
    # Save Best
    if val_psnr > best_psnr:
        best_psnr = val_psnr
        epochs_without_improvement = 0
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'best_psnr': best_psnr,
        }, os.path.join(experiment_dir, 'best_model.pth'))
        print(f"  ✓ Saved best model ({best_psnr:.2f} dB)")
    else:
        epochs_without_improvement += 1
    
    if epochs_without_improvement >= config.early_stop_patience:
        print("Early stopping triggered.")
        break

print(f"Training complete. Best PSNR: {best_psnr:.2f} dB")

In [ ]:
# 12. Visualization
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train')
plt.plot(history['val_loss'], label='Val')
plt.title('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['train_psnr'], label='Train')
plt.plot(history['val_psnr'], label='Val')
plt.title('PSNR')
plt.legend()
plt.show()

In [ ]:
# 13. Sample Predictions
# Load best model
checkpoint = torch.load(os.path.join(experiment_dir, 'best_model.pth'), map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

# Visualize
n_samples = 3
indices = random.sample(range(len(val_dataset)), min(n_samples, len(val_dataset)))

fig, axes = plt.subplots(n_samples, 3, figsize=(15, 5*n_samples))
if n_samples == 1: axes = axes.reshape(1, -1)

with torch.no_grad():
    for i, idx in enumerate(indices):
        blur, sharp = val_dataset[idx]
        blur_in = blur.unsqueeze(0).to(device)
        pred = model(blur_in).cpu().squeeze(0)
        
        psnr = calculate_psnr(pred.unsqueeze(0), sharp.unsqueeze(0)).item()
        
        axes[i, 0].imshow(blur.permute(1, 2, 0).numpy())
        axes[i, 0].set_title("Blur")
        axes[i, 1].imshow(torch.clamp(pred, 0, 1).permute(1, 2, 0).numpy())
        axes[i, 1].set_title(f"Pred (PSNR: {psnr:.2f} dB)")
        axes[i, 2].imshow(sharp.permute(1, 2, 0).numpy())
        axes[i, 2].set_title("Sharp")
        
plt.tight_layout()
plt.show()